# செமாண்டிக் கெர்னல்

இந்த குறியீட்டு மாதிரியில், [செமாண்டிக் கெர்னல்](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework-ஐ பயன்படுத்தி ஒரு அடிப்படை ஏஜென்டை உருவாக்குவீர்கள்.

இந்த மாதிரியின் நோக்கம், பின்னர் வேறு குறியீட்டு மாதிரிகளில் பல்வேறு ஏஜென்டிக் முறைகளை செயல்படுத்தும்போது பயன்படுத்தப்படும் படிகளை உங்களுக்குக் காட்டுவது.


## தேவையான பைதான் தொகுப்புகளை இறக்குமதி செய்யவும்


In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## கிளையன்ட் உருவாக்குதல்

இந்த மாதிரியில், [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ஐ பயன்படுத்தி LLM-க்கு அணுகுவோம்.

`ai_model_id` என்பது `gpt-4o-mini` என வரையறுக்கப்பட்டுள்ளது. GitHub Models சந்தையில் கிடைக்கும் மற்றொரு மாடலுக்கு மாறி, வெவ்வேறு முடிவுகளைப் பார்க்க முயற்சிக்கவும்.

GitHub Models க்கான `base_url` ஐ பயன்படுத்துவதற்கான `Azure Inference SDK` ஐ பயன்படுத்த, Semantic Kernel இல் உள்ள `OpenAIChatCompletion` இணைப்பியைப் பயன்படுத்துவோம். Semantic Kernel ஐ மற்ற மாடல் வழங்குநர்களுக்கு பயன்படுத்துவதற்கான [கிடைக்கக்கூடிய இணைப்பிகள்](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) உள்ளன.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## ஏஜென்டை உருவாக்குதல்

கீழே, `TravelAgent` எனப்படும் ஏஜென்டை உருவாக்குகிறோம்.

இந்த எடுத்துக்காட்டில், மிகவும் எளிய வழிமுறைகளை பயன்படுத்துகிறோம். இந்த வழிமுறைகளை மாற்றி, ஏஜென்ட் எப்படி வேறுபடுகிறதென்று பார்க்கலாம்.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## ஏஜென்ட்களை இயக்குவது

இப்போது, `ChatHistory`-ஐ வரையறுத்து அதில் `system_message`-ஐ சேர்த்ததன் மூலம் ஏஜென்ட்டை இயக்கலாம். நாம் முன்பே வரையறுத்த `AGENT_INSTRUCTIONS`-ஐ இங்கு பயன்படுத்துவோம்.

இவை வரையறுக்கப்பட்ட பிறகு, `user_inputs`-ஐ உருவாக்குகிறோம், இது பயனர் ஏஜென்ட்டுக்கு அனுப்பும் தகவலாக இருக்கும். இந்தச் சூழலில், இந்தச் செய்தியை `Plan me a sunny vacation` என்று அமைத்துள்ளோம்.

ஏஜென்ட்டின் பதில் எப்படி மாறுகிறது என்பதைப் பார்க்க இந்தச் செய்தியை மாற்றுவதற்கு தயங்க வேண்டாம்.


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
